In [269]:
import pandas as pd
import numpy as np
df = pd.read_csv("Churn_Modeling.csv")
df.fillna({
    'Geography': df['Geography'].mode()[0],
    'Gender': df['Gender'].mode()[0],
    'Age': df['Age'].median(),
    'CreditScore': df['CreditScore'].median(),
    'Tenure': df['Tenure'].median(),
    'Balance': df['Balance'].median(),
    'NumOfProducts': df['NumOfProducts'].mode()[0],
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': df['EstimatedSalary'].median(),
    'Exited': 0
}, inplace=True)

In [46]:
df

RowNumber  CustomerId    Surname  CreditScore Geography  Gender   Age  \
0              1    15634602   Hargrave          619    France  Female  42.0   
1              2    15647311       Hill          608     Spain  Female  41.0   
2              3    15619304       Onio          502    France  Female  42.0   
3              4    15701354       Boni          699    France  Female  39.0   
4              5    15737888   Mitchell          850     Spain  Female  43.0   
...          ...         ...        ...          ...       ...     ...   ...   
9997        9998    15584532        Liu          709    France  Female  36.0   
9998        9999    15682355  Sabbatini          772   Germany    Male  42.0   
9999        9999    15682355  Sabbatini          772   Germany    Male  42.0   
10000      10000    15628319     Walker          792    France  Female  28.0   
10001      10000    15628319     Walker          792    France  Female  28.0   

       Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0           2       0.00              1          1               1   
1           1   83807.86              1          0               1   
2           8  159660.80              3          1               0   
3           1       0.00              2          0               0   
4           2  125510.82              1          1               1   
...       ...        ...            ...        ...             ...   
9997        7       0.00              1          0               1   
9998        3   75075.31              2          1               0   
9999        3   75075.31              2          1               0   
10000       4  130142.79              1          1               0   
10001       4  130142.79              1          1               0   

       EstimatedSalary  Exited  GeographyId  GenderId  
0            101348.88       1            1         1  
1            112542.58       0            2         1  
2            113931.57       1            1         1  
3             93826.63       0            1         1  
4             79084.10       0            2         1  
...                ...     ...          ...       ...  
9997          42085.58       1            1         1  
9998          92888.52       1            3         2  
9999          92888.52       1            3         2  
10000         38190.78       0            1         1  
10001         38190.78       0            1         1  

[10002 rows x 16 columns]

In [204]:
df.columns.tolist()

['RowNumber',
 'CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [271]:
# Before Decomposition
geography_lookup = {val: i+1 for i, val in enumerate(df['Geography'].unique())}
gender_lookup = {val: i+1 for i, val in enumerate(df['Gender'].unique())}
df['GeographyId'] = df['Geography'].map(geography_lookup)
df['GenderId'] = df['Gender'].map(gender_lookup)

loan_df = df[['CustomerId', 'CreditScore', 'Balance', 'EstimatedSalary']].sample(n=3000, replace=True, random_state=42).copy()
loan_df.insert(0, 'ApplicationId', range(1, 3001))
loan_df['LoanAmount'] = (loan_df['Balance'] * 0.4 + loan_df['EstimatedSalary'] * 0.3).round(2)
loan_df['LoanStatus'] = np.where(loan_df['CreditScore'] >= 700, 'Approved',
                         np.where(loan_df['CreditScore'] >= 600, 'Under Review', 'Rejected'))
loan_df['ApplicationDate'] = pd.date_range(start='2024-01-01', periods=3000, freq='D')
loan_df = loan_df[['ApplicationId', 'CustomerId', 'LoanAmount', 'LoanStatus', 'ApplicationDate']]

customer_df = df[['CustomerId', 'Surname', 'GeographyId', 'GenderId', 'Age']].copy()
customer_df['FullName'] = customer_df['Surname'] + "_" + customer_df['CustomerId'].astype(str)
customer_df['Email'] = customer_df['Surname'].str.lower() + customer_df['CustomerId'].astype(str) + "@bank.com"
customer_df['CreatedAt'] = pd.to_datetime('2022-01-01')
customer_df = customer_df.drop_duplicates(subset=['CustomerId'])
customer_df = customer_df.drop_duplicates(subset=['Email'])
customer_df['GeographyId'] = customer_df['GeographyId'].astype(int)


account_df = df[['CustomerId', 'Tenure', 'Balance', 'NumOfProducts']].copy()
account_df['AccountType'] = 'Standard'
account_df['OpenDate'] = pd.to_datetime('2020-01-01')
account_df['Status'] = 'Active'
account_df.insert(0, 'AccountId', range(1, len(account_df) + 1))
account_df['LinkedLoanId'] = loan_df.sample(n=len(account_df), replace=True, random_state=42)['ApplicationId'].values

creditcard_df = df[['CustomerId', 'HasCrCard']].copy()
creditcard_df['CardType'] = 'Visa'
creditcard_df['CardLimit'] = 5000.0
creditcard_df['LinkedAccountId'] = account_df['AccountId'].values

activity_df = df[['CustomerId', 'IsActiveMember']].copy()
activity_df.rename(columns={'IsActiveMember': 'IsActive'}, inplace=True)
activity_df['LastLoginDate'] = pd.to_datetime('2024-05-01')
activity_df['LoginFrequency'] = 5
activity_df['LinkedLoanId'] = loan_df.sample(n=len(activity_df), replace=True, random_state=24)['ApplicationId'].values

salary_df = df[['CustomerId', 'EstimatedSalary']].copy()
salary_df['SalaryBand'] = pd.cut(salary_df['EstimatedSalary'], bins=[0, 50000, 100000, 150000, 250000],
                                 labels=['Low', 'Medium', 'High', 'Very High'])
salary_df['LastIncrement'] = 0.0
salary_df['LinkedAccountId'] = account_df['AccountId'].values

churn_df = df[['CustomerId', 'Exited']].copy()
churn_df['ChurnReason'] = np.where(df['Exited'] == 1, 'Unhappy with service', None)
churn_df['LastContactDate'] = pd.to_datetime('2024-06-01')

risk_df = df[['CustomerId', 'CreditScore']].copy()
risk_df['ScoreCategory'] = pd.cut(df['CreditScore'], bins=[300, 580, 670, 740, 800, 850],
                                  labels=['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'])
risk_df['LinkedAccountId'] = account_df['AccountId'].values

flag_df = df[['CustomerId']].copy()
flag_df['IsHighValueCustomer'] = df['Balance'] > 100000
flag_df['IsFlaggedForReview'] = (df['CreditScore'] < 500) | (df['Exited'] == 1)

customerlog_df = df[['CustomerId']].copy()
customerlog_df['Action'] = 'Account Created'
customerlog_df['ActionDate'] = pd.to_datetime('2024-06-01')
customerlog_df.insert(0, 'LogId', range(1, len(customerlog_df) + 1))
customerlog_df['LinkedLoanId'] = loan_df.sample(n=len(customerlog_df), replace=True, random_state=88)['ApplicationId'].values

for df_ in [creditcard_df, activity_df, salary_df, churn_df, risk_df, flag_df]:
    df_.drop_duplicates(subset=['CustomerId'], inplace=True)
    df_.reset_index(drop=True, inplace=True)

In [208]:
loan_df

ApplicationId  CustomerId  LoanAmount    LoanStatus ApplicationDate
7270              1    15706268    75238.46  Under Review      2024-01-01
860               2    15807663   105343.40  Under Review      2024-01-02
5390              3    15573851    27666.04      Approved      2024-01-03
5191              4    15681075      211.95  Under Review      2024-01-04
5734              5    15596647    48699.41      Approved      2024-01-05
...             ...         ...         ...           ...             ...
6416           2996    15789379   104181.28      Approved      2032-03-14
3170           2997    15791373    36164.60      Approved      2032-03-15
3835           2998    15670859    94483.41      Approved      2032-03-16
8017           2999    15631406    90471.50      Rejected      2032-03-17
2151           3000    15588614    47842.52      Approved      2032-03-18

[3000 rows x 5 columns]

In [235]:
customer_df

CustomerId    Surname  GeographyId  GenderId   Age            FullName  \
0        15634602   Hargrave            1         1  42.0   Hargrave_15634602   
1        15647311       Hill            2         1  41.0       Hill_15647311   
2        15619304       Onio            1         1  42.0       Onio_15619304   
3        15701354       Boni            1         1  39.0       Boni_15701354   
4        15737888   Mitchell            2         1  43.0   Mitchell_15737888   
...           ...        ...          ...       ...   ...                 ...   
9995     15606229   Obijiaku            1         2  39.0   Obijiaku_15606229   
9996     15569892  Johnstone            1         2  35.0  Johnstone_15569892   
9997     15584532        Liu            1         1  36.0        Liu_15584532   
9998     15682355  Sabbatini            3         2  42.0  Sabbatini_15682355   
10000    15628319     Walker            1         1  28.0     Walker_15628319   

                            Email  CreatedAt  
0       hargrave15634602@bank.com 2022-01-01  
1           hill15647311@bank.com 2022-01-01  
2           onio15619304@bank.com 2022-01-01  
3           boni15701354@bank.com 2022-01-01  
4       mitchell15737888@bank.com 2022-01-01  
...                           ...        ...  
9995    obijiaku15606229@bank.com 2022-01-01  
9996   johnstone15569892@bank.com 2022-01-01  
9997         liu15584532@bank.com 2022-01-01  
9998   sabbatini15682355@bank.com 2022-01-01  
10000     walker15628319@bank.com 2022-01-01  

[10000 rows x 8 columns]

In [212]:
account_df

AccountId  CustomerId  Tenure    Balance  NumOfProducts AccountType  \
0              1    15634602       2       0.00              1    Standard   
1              2    15647311       1   83807.86              1    Standard   
2              3    15619304       8  159660.80              3    Standard   
3              4    15701354       1       0.00              2    Standard   
4              5    15737888       2  125510.82              1    Standard   
...          ...         ...     ...        ...            ...         ...   
9997        9998    15584532       7       0.00              1    Standard   
9998        9999    15682355       3   75075.31              2    Standard   
9999       10000    15682355       3   75075.31              2    Standard   
10000      10001    15628319       4  130142.79              1    Standard   
10001      10002    15628319       4  130142.79              1    Standard   

        OpenDate  Status  LinkedLoanId  
0     2020-01-01  Active           861  
1     2020-01-01  Active          1295  
2     2020-01-01  Active          1131  
3     2020-01-01  Active          1096  
4     2020-01-01  Active          1639  
...          ...     ...           ...  
9997  2020-01-01  Active          1378  
9998  2020-01-01  Active          1178  
9999  2020-01-01  Active          1950  
10000 2020-01-01  Active           878  
10001 2020-01-01  Active          1139  

[10002 rows x 9 columns]

In [214]:
creditcard_df

CustomerId  HasCrCard CardType  CardLimit  LinkedAccountId
0       15634602        1.0     Visa     5000.0                1
1       15647311        0.0     Visa     5000.0                2
2       15619304        1.0     Visa     5000.0                3
3       15701354        0.0     Visa     5000.0                4
4       15737888        1.0     Visa     5000.0                5
...          ...        ...      ...        ...              ...
9995    15606229        1.0     Visa     5000.0             9996
9996    15569892        1.0     Visa     5000.0             9997
9997    15584532        0.0     Visa     5000.0             9998
9998    15682355        1.0     Visa     5000.0             9999
9999    15628319        1.0     Visa     5000.0            10001

[10000 rows x 5 columns]

In [216]:
activity_df

CustomerId  IsActive LastLoginDate  LoginFrequency  LinkedLoanId
0       15634602       1.0    2024-05-01               5           419
1       15647311       1.0    2024-05-01               5           900
2       15619304       0.0    2024-05-01               5          2241
3       15701354       0.0    2024-05-01               5           344
4       15737888       1.0    2024-05-01               5          2194
...          ...       ...           ...             ...           ...
9995    15606229       0.0    2024-05-01               5          1086
9996    15569892       1.0    2024-05-01               5          2120
9997    15584532       1.0    2024-05-01               5          2715
9998    15682355       0.0    2024-05-01               5           772
9999    15628319       0.0    2024-05-01               5          1914

[10000 rows x 5 columns]

In [218]:
salary_df

CustomerId  EstimatedSalary SalaryBand  LastIncrement  LinkedAccountId
0       15634602        101348.88       High            0.0                1
1       15647311        112542.58       High            0.0                2
2       15619304        113931.57       High            0.0                3
3       15701354         93826.63     Medium            0.0                4
4       15737888         79084.10     Medium            0.0                5
...          ...              ...        ...            ...              ...
9995    15606229         96270.64     Medium            0.0             9996
9996    15569892        101699.77       High            0.0             9997
9997    15584532         42085.58        Low            0.0             9998
9998    15682355         92888.52     Medium            0.0             9999
9999    15628319         38190.78        Low            0.0            10001

[10000 rows x 5 columns]

In [220]:
churn_df

CustomerId  Exited           ChurnReason LastContactDate
0       15634602       1  Unhappy with service      2024-06-01
1       15647311       0                  None      2024-06-01
2       15619304       1  Unhappy with service      2024-06-01
3       15701354       0                  None      2024-06-01
4       15737888       0                  None      2024-06-01
...          ...     ...                   ...             ...
9995    15606229       0                  None      2024-06-01
9996    15569892       0                  None      2024-06-01
9997    15584532       1  Unhappy with service      2024-06-01
9998    15682355       1  Unhappy with service      2024-06-01
9999    15628319       0                  None      2024-06-01

[10000 rows x 4 columns]

In [222]:
risk_df

CustomerId  CreditScore ScoreCategory  LinkedAccountId
0       15634602          619          Fair                1
1       15647311          608          Fair                2
2       15619304          502          Poor                3
3       15701354          699          Good                4
4       15737888          850     Excellent                5
...          ...          ...           ...              ...
9995    15606229          771     Very Good             9996
9996    15569892          516          Poor             9997
9997    15584532          709          Good             9998
9998    15682355          772     Very Good             9999
9999    15628319          792     Very Good            10001

[10000 rows x 4 columns]

In [224]:
flag_df

CustomerId  IsHighValueCustomer  IsFlaggedForReview
0       15634602                False                True
1       15647311                False               False
2       15619304                 True                True
3       15701354                False               False
4       15737888                 True               False
...          ...                  ...                 ...
9995    15606229                False               False
9996    15569892                False               False
9997    15584532                False                True
9998    15682355                False                True
9999    15628319                 True               False

[10000 rows x 3 columns]

In [226]:
customerlog_df

LogId  CustomerId           Action ActionDate  LinkedLoanId
0          1    15634602  Account Created 2024-06-01          2009
1          2    15647311  Account Created 2024-06-01          2337
2          3    15619304  Account Created 2024-06-01          2482
3          4    15701354  Account Created 2024-06-01           363
4          5    15737888  Account Created 2024-06-01          2965
...      ...         ...              ...        ...           ...
9997    9998    15584532  Account Created 2024-06-01          1781
9998    9999    15682355  Account Created 2024-06-01          2733
9999   10000    15682355  Account Created 2024-06-01           493
10000  10001    15628319  Account Created 2024-06-01           273
10001  10002    15628319  Account Created 2024-06-01          2336

[10002 rows x 5 columns]

In [273]:
# Before Decomposition
violations = []

if customer_df['CustomerId'].duplicated().any():
    violations.append("Customer.CustomerId: PRIMARY KEY constraint violated.")
if customer_df['Surname'].isnull().any():
    violations.append("Customer.Surname: NOT NULL constraint violated.")
if customer_df['Email'].duplicated().any():
    violations.append("Customer.Email: UNIQUE constraint violated.")
if (customer_df['Age'] < 18).any():
    violations.append("Customer.Age: CHECK constraint violated (Age < 18).")
if customer_df['GeographyId'].isnull().any():
    violations.append("Customer.GeographyId: NOT NULL constraint violated")
if customer_df['GenderId'].isnull().any():
    violations.append("Customer.GenderId: NOT NULL constraint violated.")
if customer_df['CreatedAt'].isnull().any():
    violations.append("Customer.CreatedAt: DEFAULT CURRENT_TIMESTAMP not applied properly.")


if loan_df['ApplicationId'].duplicated().any():
    violations.append("LoanApplication.ApplicationId: PRIMARY KEY constraint violated.")
if loan_df['CustomerId'].isnull().any():
    violations.append("LoanApplication.CustomerId: FOREIGN KEY NULL.")
if not loan_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("LoanApplication.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if loan_df['LoanAmount'].isnull().any():
    violations.append("LoanApplication.LoanAmount: NOT NULL constraint violated.")
if loan_df['LoanStatus'].isnull().any():
    violations.append("LoanApplication.LoanStatus: DEFAULT 'Pending' not applied properly.")
if loan_df['ApplicationDate'].isnull().any():
    violations.append("LoanApplication.ApplicationDate: DEFAULT CURRENT_DATE not applied properly.")


if account_df['AccountId'].duplicated().any():
    violations.append("Account.AccountId: PRIMARY KEY constraint violated.")
if account_df['CustomerId'].isnull().any():
    violations.append("Account.CustomerId: FOREIGN KEY NULL.")
if not account_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("Account.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if (account_df['Tenure'] < 0).any():
    violations.append("Account.Tenure: CHECK constraint violated (Tenure < 0).")
if (account_df['NumOfProducts'] < 1).any():
    violations.append("Account.NumOfProducts: CHECK constraint violated (NumOfProducts < 1).")
if account_df['OpenDate'].isnull().any():
    violations.append("Account.OpenDate: DEFAULT CURRENT_DATE not applied properly.")
if account_df['Status'].isnull().any():
    violations.append("Account.Status: DEFAULT 'Active' not applied properly.")
if account_df['AccountType'].isnull().any():
    violations.append("Account.AccountType: DEFAULT 'Standard' not applied properly.")
if account_df['LinkedLoanId'].isnull().any():
    violations.append("Account.LinkedLoanId: FOREIGN KEY REFERENCES LoanApplication(ApplicationId) not applied properly.")


if creditcard_df['CustomerId'].duplicated().any():
    violations.append("CreditCard.CustomerId: PRIMARY KEY constraint violated.")
if not creditcard_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("CreditCard.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if creditcard_df['HasCrCard'].isnull().any():
    violations.append("CreditCard.HasCrCard: DEFAULT FALSE not applied properly.")
if creditcard_df['CardType'].isnull().any():
    violations.append("CreditCard.CardType: DEFAULT 'Visa' not applied properly.")
if creditcard_df['CardLimit'].isnull().any():
    violations.append("CreditCard.CardLimit: DEFAULT 5000.00 not applied properly.")
if creditcard_df['LinkedAccountId'].isnull().any():
    violations.append("CreditCard.LinkedAccountId: FOREIGN KEY REFERENCES Account(AccountId) not applied properly.")


if activity_df['CustomerId'].duplicated().any():
    violations.append("ActivityStatus.CustomerId: PRIMARY KEY constraint violated.")
if not activity_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("ActivityStatus.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if activity_df['IsActive'].isnull().any():
    violations.append("ActivityStatus.IsActive: DEFAULT FALSE not applied properly.")
if activity_df['LoginFrequency'].isnull().any():
    violations.append("ActivityStatus.LoginFrequency: DEFAULT 0 not applied properly.")
if activity_df['LinkedLoanId'].isnull().any():
    violations.append("ActivityStatus.LinkedLoanId: FOREIGN KEY REFERENCES LoanApplication(ApplicationId) not applied properly.")


if salary_df['CustomerId'].duplicated().any():
    violations.append("Salary.CustomerId: PRIMARY KEY constraint violated.")
if not salary_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("Salary.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if salary_df['EstimatedSalary'].isnull().any():
    violations.append("Salary.EstimatedSalary: NOT NULL constraint violated.")
if salary_df['LastIncrement'].isnull().any():
    violations.append("Salary.LastIncrement: DEFAULT 0.00 not applied properly.")
if salary_df['LinkedAccountId'].isnull().any():
    violations.append("Salary.LinkedAccountId: FOREIGN KEY REFERENCES Account(AccountId) not applied properly.")


if churn_df['CustomerId'].duplicated().any():
    violations.append("ChurnStatus.CustomerId: PRIMARY KEY constraint violated.")
if not churn_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("ChurnStatus.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if churn_df['Exited'].isnull().any():
    violations.append("ChurnStatus.Exited: NOT NULL constraint violated.")


if customerlog_df['LogId'].duplicated().any():
    violations.append("CustomerLog.LogId: PRIMARY KEY constraint violated.")
if customerlog_df['CustomerId'].isnull().any():
    violations.append("CustomerLog.CustomerId: FOREIGN KEY NULL.")
if not customerlog_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("CustomerLog.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if customerlog_df['ActionDate'].isnull().any():
    violations.append("CustomerLog.ActionDate: DEFAULT CURRENT_TIMESTAMP not applied properly.")
if customerlog_df['Action'].isnull().any():
    violations.append("CustomerLog.Action: NULL value found in Action column.")
if customerlog_df['LinkedLoanId'].isnull().any():
    violations.append("CustomerLog.LinkedLoanId: FOREIGN KEY REFERENCES LoanApplication(ApplicationId) not applied properly.")


if risk_df['CustomerId'].duplicated().any():
    violations.append("RiskScore.CustomerId: PRIMARY KEY constraint violated.")
if not risk_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("RiskScore.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if ((risk_df['CreditScore'] < 300) | (risk_df['CreditScore'] > 850)).any():
    violations.append("RiskScore.CreditScore: CHECK constraint violated (not between 300 and 850).")
if risk_df['LinkedAccountId'].isnull().any():
    violations.append("RiskScore.LinkedAccountId: FOREIGN KEY REFERENCES Account(AccountId) not applied properly.")


if flag_df['CustomerId'].duplicated().any():
    violations.append("StatusFlag.CustomerId: PRIMARY KEY constraint violated.")
if not flag_df['CustomerId'].isin(customer_df['CustomerId']).all():
    violations.append("StatusFlag.CustomerId: FOREIGN KEY REFERENCES Customer(CustomerId) violated.")
if flag_df['IsHighValueCustomer'].isnull().any():
    violations.append("StatusFlag.IsHighValueCustomer: DEFAULT FALSE not applied properly.")
if flag_df['IsFlaggedForReview'].isnull().any():
    violations.append("StatusFlag.IsFlaggedForReview: DEFAULT FALSE not applied properly.")

if violations:
    print(" Violations Found:\n")
    for v in violations:
        print(f" - {v}")
else:
    print(" All constraints are validated successfully.")

 All constraints are validated successfully.


In [275]:
# Before Decomposition
violations = []

if not pd.api.types.is_integer_dtype(customer_df['CustomerId']):
    violations.append("Customer.CustomerId must be BIGINT")
if not pd.api.types.is_string_dtype(customer_df['Surname']):
    violations.append("Customer.Surname must be VARCHAR")
if not pd.api.types.is_string_dtype(customer_df['FullName']):
    violations.append("Customer.FullName must be VARCHAR")
if not pd.api.types.is_string_dtype(customer_df['Email']):
    violations.append("Customer.Email must be VARCHAR")
if not pd.api.types.is_integer_dtype(customer_df['GeographyId']):
    violations.append("Customer.GeographyId must be INT")
if not pd.api.types.is_integer_dtype(customer_df['GenderId']):
    violations.append("Customer.GenderId must be INT")
if not pd.api.types.is_integer_dtype(customer_df['Age']):
    violations.append("Customer.Age must be INT")
if not pd.api.types.is_datetime64_any_dtype(customer_df['CreatedAt']):
    violations.append("Customer.CreatedAt must be TIMESTAMP")

if not pd.api.types.is_integer_dtype(loan_df['ApplicationId']):
    violations.append("LoanApplication.ApplicationId must be SERIAL/INT")
if not pd.api.types.is_integer_dtype(loan_df['CustomerId']):
    violations.append("LoanApplication.CustomerId must be BIGINT")
if not pd.api.types.is_float_dtype(loan_df['LoanAmount']):
    violations.append("LoanApplication.LoanAmount must be DECIMAL(10,2)")
if not pd.api.types.is_string_dtype(loan_df['LoanStatus']):
    violations.append("LoanApplication.LoanStatus must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(loan_df['ApplicationDate']):
    violations.append("LoanApplication.ApplicationDate must be DATE")

if not pd.api.types.is_integer_dtype(account_df['AccountId']):
    violations.append("Account.AccountId must be SERIAL/INT")
if not pd.api.types.is_integer_dtype(account_df['CustomerId']):
    violations.append("Account.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(account_df['LinkedLoanId']):
    violations.append("Account.LinkedLoanId must be INT")
if not pd.api.types.is_integer_dtype(account_df['Tenure']):
    violations.append("Account.Tenure must be INT")
if not pd.api.types.is_float_dtype(account_df['Balance']):
    violations.append("Account.Balance must be DECIMAL(15,2)")
if not pd.api.types.is_integer_dtype(account_df['NumOfProducts']):
    violations.append("Account.NumOfProducts must be INT")
if not pd.api.types.is_string_dtype(account_df['AccountType']):
    violations.append("Account.AccountType must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(account_df['OpenDate']):
    violations.append("Account.OpenDate must be DATE")
if not pd.api.types.is_string_dtype(account_df['Status']):
    violations.append("Account.Status must be VARCHAR")

if not pd.api.types.is_integer_dtype(creditcard_df['CustomerId']):
    violations.append("CreditCard.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(creditcard_df['LinkedAccountId']):
    violations.append("CreditCard.LinkedAccountId must be INT")
if not pd.api.types.is_bool_dtype(creditcard_df['HasCrCard']):
    violations.append("CreditCard.HasCrCard must be BOOLEAN")
if not pd.api.types.is_string_dtype(creditcard_df['CardType']):
    violations.append("CreditCard.CardType must be VARCHAR")
if not pd.api.types.is_float_dtype(creditcard_df['CardLimit']):
    violations.append("CreditCard.CardLimit must be DECIMAL")

if not pd.api.types.is_integer_dtype(activity_df['CustomerId']):
    violations.append("ActivityStatus.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(activity_df['LinkedLoanId']):
    violations.append("ActivityStatus.LinkedLoanId must be INT")
if not pd.api.types.is_bool_dtype(activity_df['IsActive']):
    violations.append("ActivityStatus.IsActive must be BOOLEAN")
if not pd.api.types.is_datetime64_any_dtype(activity_df['LastLoginDate']):
    violations.append("ActivityStatus.LastLoginDate must be DATE")
if not pd.api.types.is_integer_dtype(activity_df['LoginFrequency']):
    violations.append("ActivityStatus.LoginFrequency must be INT")

if not pd.api.types.is_integer_dtype(salary_df['CustomerId']):
    violations.append("Salary.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(salary_df['LinkedAccountId']):
    violations.append("Salary.LinkedAccountId must be INT")
if not pd.api.types.is_float_dtype(salary_df['EstimatedSalary']):
    violations.append("Salary.EstimatedSalary must be DECIMAL")
if not pd.api.types.is_string_dtype(salary_df['SalaryBand']):
    violations.append("Salary.SalaryBand must be VARCHAR")
if not pd.api.types.is_float_dtype(salary_df['LastIncrement']):
    violations.append("Salary.LastIncrement must be DECIMAL")

if not pd.api.types.is_integer_dtype(churn_df['CustomerId']):
    violations.append("ChurnStatus.CustomerId must be BIGINT")
if not pd.api.types.is_bool_dtype(churn_df['Exited']):
    violations.append("ChurnStatus.Exited must be BOOLEAN")
if not pd.api.types.is_string_dtype(churn_df['ChurnReason']):
    violations.append("ChurnStatus.ChurnReason must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(churn_df['LastContactDate']):
    violations.append("ChurnStatus.LastContactDate must be DATE")

if not pd.api.types.is_integer_dtype(risk_df['CustomerId']):
    violations.append("RiskScore.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(risk_df['LinkedAccountId']):
    violations.append("RiskScore.LinkedAccountId must be INT")
if not pd.api.types.is_integer_dtype(risk_df['CreditScore']):
    violations.append("RiskScore.CreditScore must be INT")
if not pd.api.types.is_string_dtype(risk_df['ScoreCategory']):
    violations.append("RiskScore.ScoreCategory must be VARCHAR")

if not pd.api.types.is_integer_dtype(flag_df['CustomerId']):
    violations.append("StatusFlag.CustomerId must be BIGINT")
if not pd.api.types.is_bool_dtype(flag_df['IsHighValueCustomer']):
    violations.append("StatusFlag.IsHighValueCustomer must be BOOLEAN")
if not pd.api.types.is_bool_dtype(flag_df['IsFlaggedForReview']):
    violations.append("StatusFlag.IsFlaggedForReview must be BOOLEAN")

if not pd.api.types.is_integer_dtype(customerlog_df['LogId']):
    violations.append("CustomerLog.LogId must be SERIAL/INT")
if not pd.api.types.is_integer_dtype(customerlog_df['CustomerId']):
    violations.append("CustomerLog.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(customerlog_df['LinkedLoanId']):
    violations.append("CustomerLog.LinkedLoanId must be INT")
if not pd.api.types.is_string_dtype(customerlog_df['Action']):
    violations.append("CustomerLog.Action must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(customerlog_df['ActionDate']):
    violations.append("CustomerLog.ActionDate must be TIMESTAMP")

if violations:
    print("\n Data Type Violations Found:\n")
    for v in violations:
        print(f"- {v}")
else:
    print(" All data types are valid and match the CREATE TABLE schema.")


 Data Type Violations Found:

- Customer.Age must be INT
- CreditCard.HasCrCard must be BOOLEAN
- ActivityStatus.IsActive must be BOOLEAN
- ChurnStatus.Exited must be BOOLEAN
- ChurnStatus.ChurnReason must be VARCHAR


In [277]:
customer_df['Age'] = customer_df['Age'].astype(int)
creditcard_df['HasCrCard'] = creditcard_df['HasCrCard'].apply(lambda x: True if str(x).lower() in ['1', 'true'] else False)
activity_df['IsActive'] = activity_df['IsActive'].apply(lambda x: True if str(x).lower() in ['1', 'true'] else False)
churn_df['Exited'] = churn_df['Exited'].apply(lambda x: True if str(x).lower() in ['1', 'true'] else False)
churn_df['ChurnReason'] = churn_df['ChurnReason'].fillna('').astype(str)

In [279]:
# Before Decomposition
violations = []

if not pd.api.types.is_integer_dtype(customer_df['CustomerId']):
    violations.append("Customer.CustomerId must be BIGINT")
if not pd.api.types.is_string_dtype(customer_df['Surname']):
    violations.append("Customer.Surname must be VARCHAR")
if not pd.api.types.is_string_dtype(customer_df['FullName']):
    violations.append("Customer.FullName must be VARCHAR")
if not pd.api.types.is_string_dtype(customer_df['Email']):
    violations.append("Customer.Email must be VARCHAR")
if not pd.api.types.is_integer_dtype(customer_df['GeographyId']):
    violations.append("Customer.GeographyId must be INT")
if not pd.api.types.is_integer_dtype(customer_df['GenderId']):
    violations.append("Customer.GenderId must be INT")
if not pd.api.types.is_integer_dtype(customer_df['Age']):
    violations.append("Customer.Age must be INT")
if not pd.api.types.is_datetime64_any_dtype(customer_df['CreatedAt']):
    violations.append("Customer.CreatedAt must be TIMESTAMP")

if not pd.api.types.is_integer_dtype(loan_df['ApplicationId']):
    violations.append("LoanApplication.ApplicationId must be SERIAL/INT")
if not pd.api.types.is_integer_dtype(loan_df['CustomerId']):
    violations.append("LoanApplication.CustomerId must be BIGINT")
if not pd.api.types.is_float_dtype(loan_df['LoanAmount']):
    violations.append("LoanApplication.LoanAmount must be DECIMAL(10,2)")
if not pd.api.types.is_string_dtype(loan_df['LoanStatus']):
    violations.append("LoanApplication.LoanStatus must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(loan_df['ApplicationDate']):
    violations.append("LoanApplication.ApplicationDate must be DATE")

if not pd.api.types.is_integer_dtype(account_df['AccountId']):
    violations.append("Account.AccountId must be SERIAL/INT")
if not pd.api.types.is_integer_dtype(account_df['CustomerId']):
    violations.append("Account.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(account_df['LinkedLoanId']):
    violations.append("Account.LinkedLoanId must be INT")
if not pd.api.types.is_integer_dtype(account_df['Tenure']):
    violations.append("Account.Tenure must be INT")
if not pd.api.types.is_float_dtype(account_df['Balance']):
    violations.append("Account.Balance must be DECIMAL(15,2)")
if not pd.api.types.is_integer_dtype(account_df['NumOfProducts']):
    violations.append("Account.NumOfProducts must be INT")
if not pd.api.types.is_string_dtype(account_df['AccountType']):
    violations.append("Account.AccountType must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(account_df['OpenDate']):
    violations.append("Account.OpenDate must be DATE")
if not pd.api.types.is_string_dtype(account_df['Status']):
    violations.append("Account.Status must be VARCHAR")

if not pd.api.types.is_integer_dtype(creditcard_df['CustomerId']):
    violations.append("CreditCard.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(creditcard_df['LinkedAccountId']):
    violations.append("CreditCard.LinkedAccountId must be INT")
if not pd.api.types.is_bool_dtype(creditcard_df['HasCrCard']):
    violations.append("CreditCard.HasCrCard must be BOOLEAN")
if not pd.api.types.is_string_dtype(creditcard_df['CardType']):
    violations.append("CreditCard.CardType must be VARCHAR")
if not pd.api.types.is_float_dtype(creditcard_df['CardLimit']):
    violations.append("CreditCard.CardLimit must be DECIMAL")

if not pd.api.types.is_integer_dtype(activity_df['CustomerId']):
    violations.append("ActivityStatus.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(activity_df['LinkedLoanId']):
    violations.append("ActivityStatus.LinkedLoanId must be INT")
if not pd.api.types.is_bool_dtype(activity_df['IsActive']):
    violations.append("ActivityStatus.IsActive must be BOOLEAN")
if not pd.api.types.is_datetime64_any_dtype(activity_df['LastLoginDate']):
    violations.append("ActivityStatus.LastLoginDate must be DATE")
if not pd.api.types.is_integer_dtype(activity_df['LoginFrequency']):
    violations.append("ActivityStatus.LoginFrequency must be INT")

if not pd.api.types.is_integer_dtype(salary_df['CustomerId']):
    violations.append("Salary.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(salary_df['LinkedAccountId']):
    violations.append("Salary.LinkedAccountId must be INT")
if not pd.api.types.is_float_dtype(salary_df['EstimatedSalary']):
    violations.append("Salary.EstimatedSalary must be DECIMAL")
if not pd.api.types.is_string_dtype(salary_df['SalaryBand']):
    violations.append("Salary.SalaryBand must be VARCHAR")
if not pd.api.types.is_float_dtype(salary_df['LastIncrement']):
    violations.append("Salary.LastIncrement must be DECIMAL")

if not pd.api.types.is_integer_dtype(churn_df['CustomerId']):
    violations.append("ChurnStatus.CustomerId must be BIGINT")
if not pd.api.types.is_bool_dtype(churn_df['Exited']):
    violations.append("ChurnStatus.Exited must be BOOLEAN")
if not pd.api.types.is_string_dtype(churn_df['ChurnReason']):
    violations.append("ChurnStatus.ChurnReason must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(churn_df['LastContactDate']):
    violations.append("ChurnStatus.LastContactDate must be DATE")

if not pd.api.types.is_integer_dtype(risk_df['CustomerId']):
    violations.append("RiskScore.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(risk_df['LinkedAccountId']):
    violations.append("RiskScore.LinkedAccountId must be INT")
if not pd.api.types.is_integer_dtype(risk_df['CreditScore']):
    violations.append("RiskScore.CreditScore must be INT")
if not pd.api.types.is_string_dtype(risk_df['ScoreCategory']):
    violations.append("RiskScore.ScoreCategory must be VARCHAR")

if not pd.api.types.is_integer_dtype(flag_df['CustomerId']):
    violations.append("StatusFlag.CustomerId must be BIGINT")
if not pd.api.types.is_bool_dtype(flag_df['IsHighValueCustomer']):
    violations.append("StatusFlag.IsHighValueCustomer must be BOOLEAN")
if not pd.api.types.is_bool_dtype(flag_df['IsFlaggedForReview']):
    violations.append("StatusFlag.IsFlaggedForReview must be BOOLEAN")

if not pd.api.types.is_integer_dtype(customerlog_df['LogId']):
    violations.append("CustomerLog.LogId must be SERIAL/INT")
if not pd.api.types.is_integer_dtype(customerlog_df['CustomerId']):
    violations.append("CustomerLog.CustomerId must be BIGINT")
if not pd.api.types.is_integer_dtype(customerlog_df['LinkedLoanId']):
    violations.append("CustomerLog.LinkedLoanId must be INT")
if not pd.api.types.is_string_dtype(customerlog_df['Action']):
    violations.append("CustomerLog.Action must be VARCHAR")
if not pd.api.types.is_datetime64_any_dtype(customerlog_df['ActionDate']):
    violations.append("CustomerLog.ActionDate must be TIMESTAMP")

if violations:
    print("\n Data Type Violations Found:\n")
    for v in violations:
        print(f"- {v}")
else:
    print(" All data types are valid and match the CREATE TABLE schema.")

 All data types are valid and match the CREATE TABLE schema.


In [281]:
customer_df.to_csv("Customer.csv", index=False)
loan_df.to_csv("LoanApplication.csv", index=False)
account_df.to_csv("Account.csv", index=False)
creditcard_df.to_csv("CreditCard.csv", index=False)
activity_df.to_csv("ActivityStatus.csv", index=False)
salary_df.to_csv("Salary.csv", index=False)
churn_df.to_csv("ChurnStatus.csv", index=False)
risk_df.to_csv("RiskScore.csv", index=False)
flag_df.to_csv("StatusFlag.csv", index=False)
customerlog_df.to_csv("CustomerLog.csv", index=False)

In [267]:
# Decomposed Tables 

salary_df = pd.read_csv("Salary.csv")
risk_df = pd.read_csv("RiskScore.csv")

salary_band_mapping = salary_df[['EstimatedSalary', 'SalaryBand']].drop_duplicates().dropna()
salary_details = salary_df.drop(columns=['SalaryBand'])

score_category_mapping = risk_df[['CreditScore', 'ScoreCategory']].drop_duplicates().dropna()
risk_score_details = risk_df.drop(columns=['ScoreCategory'])

salary_band_mapping.to_csv("SalaryBandMapping.csv", index=False)
salary_details.to_csv("SalaryDetails.csv", index=False)
score_category_mapping.to_csv("ScoreCategoryMapping.csv", index=False)
risk_score_details.to_csv("RiskScoreDetails.csv", index=False)


In [265]:
# Decomposed Tables
violations = []

salary_band_mapping = pd.read_csv("SalaryBandMapping.csv")
salary_details = pd.read_csv("SalaryDetails.csv")
score_category_mapping = pd.read_csv("ScoreCategoryMapping.csv")
risk_score_details = pd.read_csv("RiskScoreDetails.csv")


if not pd.api.types.is_float_dtype(salary_band_mapping['EstimatedSalary']):
    violations.append("SalaryBandMapping.EstimatedSalary must be DECIMAL(15,2)")
if salary_band_mapping['EstimatedSalary'].duplicated().any():
    violations.append("SalaryBandMapping.EstimatedSalary must be UNIQUE (Primary Key)")
if salary_band_mapping['EstimatedSalary'].isnull().any():
    violations.append("SalaryBandMapping.EstimatedSalary cannot be NULL")
if salary_band_mapping['SalaryBand'].isnull().any():
    violations.append("SalaryBandMapping.SalaryBand cannot be NULL")
if not pd.api.types.is_string_dtype(salary_band_mapping['SalaryBand']):
    violations.append("SalaryBandMapping.SalaryBand must be VARCHAR(30)")


if not pd.api.types.is_integer_dtype(salary_details['CustomerId']):
    violations.append("SalaryDetails.CustomerId must be BIGINT")
if salary_details['CustomerId'].duplicated().any():
    violations.append("SalaryDetails.CustomerId must be UNIQUE (Primary Key)")
if salary_details['CustomerId'].isnull().any():
    violations.append("SalaryDetails.CustomerId cannot be NULL")

if salary_details['LinkedAccountId'].isnull().any():
    violations.append("SalaryDetails.LinkedAccountId cannot be NULL")  # Optional FK, so this can be relaxed if NULL allowed

if not pd.api.types.is_float_dtype(salary_details['EstimatedSalary']):
    violations.append("SalaryDetails.EstimatedSalary must be DECIMAL(15,2)")
if salary_details['EstimatedSalary'].isnull().any():
    violations.append("SalaryDetails.EstimatedSalary cannot be NULL")

if not pd.api.types.is_float_dtype(salary_details['LastIncrement']):
    violations.append("SalaryDetails.LastIncrement must be DECIMAL(10,2)")


invalid_salaries = salary_details[~salary_details['EstimatedSalary'].isin(salary_band_mapping['EstimatedSalary'])]
if not invalid_salaries.empty:
    violations.append("SalaryDetails.EstimatedSalary must exist in SalaryBandMapping (Foreign Key violation)")


if not pd.api.types.is_integer_dtype(score_category_mapping['CreditScore']):
    violations.append("ScoreCategoryMapping.CreditScore must be INT")
if score_category_mapping['CreditScore'].duplicated().any():
    violations.append("ScoreCategoryMapping.CreditScore must be UNIQUE (Primary Key)")
if score_category_mapping['CreditScore'].isnull().any():
    violations.append("ScoreCategoryMapping.CreditScore cannot be NULL")
if not score_category_mapping['CreditScore'].between(300, 850).all():
    violations.append("ScoreCategoryMapping.CreditScore must be between 300 and 850 (CHECK constraint)")

if score_category_mapping['ScoreCategory'].isnull().any():
    violations.append("ScoreCategoryMapping.ScoreCategory cannot be NULL")
if not pd.api.types.is_string_dtype(score_category_mapping['ScoreCategory']):
    violations.append("ScoreCategoryMapping.ScoreCategory must be VARCHAR(50)")


if not pd.api.types.is_integer_dtype(risk_score_details['CustomerId']):
    violations.append("RiskScoreDetails.CustomerId must be BIGINT")
if risk_score_details['CustomerId'].duplicated().any():
    violations.append("RiskScoreDetails.CustomerId must be UNIQUE (Primary Key)")
if risk_score_details['CustomerId'].isnull().any():
    violations.append("RiskScoreDetails.CustomerId cannot be NULL")

if risk_score_details['LinkedAccountId'].isnull().any():
    violations.append("RiskScoreDetails.LinkedAccountId cannot be NULL")  # Optional FK, so can be relaxed if needed

if not pd.api.types.is_integer_dtype(risk_score_details['CreditScore']):
    violations.append("RiskScoreDetails.CreditScore must be INT")
if risk_score_details['CreditScore'].isnull().any():
    violations.append("RiskScoreDetails.CreditScore cannot be NULL")
if not risk_score_details['CreditScore'].between(300, 850).all():
    violations.append("RiskScoreDetails.CreditScore must be between 300 and 850 (CHECK constraint)")


invalid_scores = risk_score_details[~risk_score_details['CreditScore'].isin(score_category_mapping['CreditScore'])]
if not invalid_scores.empty:
    violations.append("RiskScoreDetails.CreditScore must exist in ScoreCategoryMapping (Foreign Key violation)")

if violations:
    print(" Violations Detected:\n")
    for v in violations:
        print("-", v)
else:
    print(" All data types and constraints are satisfied.")


 All data types and constraints are satisfied.


In [283]:
salary_details

CustomerId  EstimatedSalary  LastIncrement  LinkedAccountId
0       15634602        101348.88            0.0                1
1       15647311        112542.58            0.0                2
2       15619304        113931.57            0.0                3
3       15701354         93826.63            0.0                4
4       15737888         79084.10            0.0                5
...          ...              ...            ...              ...
9995    15606229         96270.64            0.0             9996
9996    15569892        101699.77            0.0             9997
9997    15584532         42085.58            0.0             9998
9998    15682355         92888.52            0.0             9999
9999    15628319         38190.78            0.0            10001

[10000 rows x 4 columns]

In [285]:
risk_score_details

CustomerId  CreditScore  LinkedAccountId
0       15634602          619                1
1       15647311          608                2
2       15619304          502                3
3       15701354          699                4
4       15737888          850                5
...          ...          ...              ...
9995    15606229          771             9996
9996    15569892          516             9997
9997    15584532          709             9998
9998    15682355          772             9999
9999    15628319          792            10001

[10000 rows x 3 columns]